In [1]:
from z3 import *

We are going to verify this program:
```
assume(B > 0);
C = A;
D = 0;
while (D < B) {
    C++; D++;
}
assert(C == A + B);
```

In [2]:
_int = IntSort(); _bool = BoolSort()
Inv = Function("Inv", _int, _int, _int, _int, _bool)
A, B, C, D, C1, D1 = Ints("A B C D C1 D1")

PRECONDITION = And(B > 0, C == A, D == 0)
POSTCONDITION = (C == A + B)

In [3]:
s = SolverFor('HORN')

In [4]:
# B > 0 ∧ C = A ∧ D = 0 → Inv
s.add(ForAll([A,B,C,D], Implies(PRECONDITION, 
                                Inv(A, B, C, D))))

In [5]:
# Inv ∧ D < B ∧ C' = C + 1 ∧ D' = D + 1 → Inv
s.add(ForAll([A,B,C,D,C1,D1], Implies(
    And(Inv(A, B, C, D), D < B, C1 == C + 1, D1 == D + 1),
    Inv(A, B, C1, D1)
)))

In [6]:
# Inv ∧ ¬(D < B) → C = A + B
s.add(ForAll([A,B,C,D], Implies(
    And(Inv(A,B,C,D), D >= B, Not(POSTCONDITION)),
                               False)))

In [7]:
s.check()

sat

In [8]:
s.model()

[Inv = [else ->
        And(Var(2) + -1*Var(0) + -1*Var(3) <= 0,
            Var(0) + Var(3) + -1*Var(2) <= 0,
            Var(2) + -1*Var(0) + -1*Var(1) <= 0)]]

This is the loop invariant!

What is means is:

$C = A + D \land C \leq A + B$

Make sure you understand why. Notice that $v_0$ represents the variable A, $v_1$ is B and so on.

Can you find a simpler (slightly shorter) invariant?